In [2]:
import plotly.express as px
import numpy as np
import pandas as pd
from tqdm import tqdm

---

# 1. 데이터 불러오기 

In [3]:
data = pd.read_csv('data/clustered_data.csv', encoding='unicode_escape')
topic_data = pd.read_csv('data/topic_per_doc.csv')

In [4]:
# 클러스터의 null값을 0으로 채워주기 
data['Clsuter=40'].fillna(0, inplace=True)

---

# 2. 문서별 토픽 할당 

## 2-1. 토픽 분포 확인 

In [ ]:
topic_data.iloc[0, :].describe()

In [ ]:
for doc_idx in range(len(data)-1) :
    print('0.11이상의 토픽 할당도 갯수 : {}'.format(sum(topic_data.iloc[doc_idx, :] > 0.1)))

- 특정 문서에서 0.01 이상 할당된 토픽은 해당 문서의 토픽이라고 보기로 결정 
- 0.1로 할시 대부분 1-2의 값으로 너무 적은 토픽 갯수 할당
- 0.05로 할 경우 3-7개의 적당한 토픽 수를 가지나 몇몇 문서는 0개의 토픽을 가지므로 부적절 <br></br>

- 추가
    - 0.01로 진행 시 클러스터별로 토픽의 수가 너무 많음
    - 0.05나 0.1로 조정해야할 필요를 느낌 

## 2-2. 토픽 할당 

In [ ]:
# 문서별 토픽 딕셔너리 만들기 
doc_topic_dict = {}

for row_idx, row in tqdm(topic_data.iterrows()) :
    topic_percentage = {}
    meaning_elements = row[row>0.01]
    
    for i, v in meaning_elements.items() :
        topic_percentage[int(i[5:])] = v
    doc_topic_dict[row_idx] = topic_percentage

In [ ]:
# 트리맵 구현용 데이터프레임 만들기 
df = []

for k1, dict_ in doc_topic_dict.items() :
    for k2, v2 in dict_.items() :
        row = []
        row.append(k1)
        row.append(k2)
        row.append(data.loc[k1, 'Clsuter=40'])
        row.append('project')
        row.append(v2)
        row.append(0)
        df.append(row)

df = pd.DataFrame(df, columns=['doc_index', 'topic', 'cluster', 'project', 'topic_percentage', 'cluster_count'])

# 클러스터의 null값 40으로 채우기 
df.cluster.fillna(40, inplace=True)

In [ ]:
# 클러스터 갯수 구하기
cluster_list = list(data['Clsuter=40'].fillna(40))
cluster_count_dict = {}

for cluster in cluster_list: 
    cluster_count_dict[int(cluster)] = cluster_list.count(cluster)

cluster_count_dict = {k : v for k, v in cluster_count_dict.items()}

# df에 클러스터 갯수 데이터 입력 

# 클러스터명 형식 변경
# 기존 float형
# 변경 'cluster' + str(int형)

for row_idx in tqdm(range(len(df))) :
    df.loc[row_idx, 'cluster_count'] = cluster_count_dict[int(df.loc[row_idx, 'cluster'])]
    df.loc[row_idx, 'cluster'] = 'cluster' + str(int(df.loc[row_idx, 'cluster']))

## 2-3. 클러스터-토픽 행렬 만들기 

- 클러스터별 토픽의 비중을 파악하기 위해 클러스터-토픽 행렬을 만들기 
- 토픽-문서 벡터 : topic_data
- 클러스터-문서 벡터 : data 변수 사용하여 행렬 만들기 


In [4]:
cluster_data = np.zeros((len(data), 40))

for idx, value in enumerate(data['Clsuter=40']) :
    cluster_data[idx, int(value)] += 1

---

# 3. Tree map 시각화

In [ ]:
fig = px.treemap(df, path=['project', 'cluster', 'topic'],
                values='topic_percentage')
                
fig.show()

---

# 4. Dash 연동

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [ ]:
app = dash.Dash()
app.layout = html.Div([
                dcc.Graph(figure=fig)
])

In [ ]:
app.run_server(debug=True, use_reloader=False)

---

# 5. Chart studio 활용하여 웹용 링크 만들기

In [ ]:
import chart_studio
from chart_studio.plotly import plot, iplot

In [ ]:
chart_studio.tools.set_credentials_file(username='injokim', api_key='ZAsiIjcDIzXdYHa8CXbw')

In [ ]:
plot(fig, filename = 'treemap', auto_open=True)

---

# 6. 질문 사항

## 6-1. 클러스터 박스의 크기가 실제 클러스터의 비율을 의미할까요?
- 본 treemap에서 사용된 데이터는 문서별 토픽 비율 하나밖에 없음  
- 클러스터의 비율이 시각화에 반영되지 않았을 가능성이 있음 <br></br>

- 시각화 결과 cluster 4, 3, 33이 가장 큰 크기를 차지함
- 실제 비율 확인 결과 해당 클러스터들이 높은 비중을 차지하는지 확인해보기 

In [13]:
counter = []
for i in range(40) :
   count_ = []
   count_.append(i)
   count_.append(list(data['Clsuter=40']).count(i))
   counter.append(count_) 

counter = pd.DataFrame(counter, columns=['index', 'count'])

In [17]:
counter.sort_values(by=['count'], ascending=False).head()

,index,count
4,4,242
33,33,169
3,3,158
22,22,157
8,8,148


- 확인 결과, 실제 분포에서도 4번, 33번, 3번등 시각화에서 두드러졌던 클러스터들의 갯수가 많은 것을 확인 
- 그냥 그대로 시각화하면 될듯함 